<a href="https://colab.research.google.com/github/tylerlum/fake-news-classifier/blob/master/BERT_End_to_End_(Fine_tuning_%2B_Predicting)_with_Cloud_TPU_Sentence_and_Sentence_Pair_Classification_Tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# BERT End to End (Fine-tuning + Predicting) in 5 minutes with Cloud TPU

## Overview

**BERT**, or **B**idirectional **E**mbedding **R**epresentations from **T**ransformers, is a new method of pre-training language representations which obtains state-of-the-art results on a wide array of Natural Language Processing (NLP) tasks. The academic paper can be found here: https://arxiv.org/abs/1810.04805.

This Colab demonstates using a free Colab Cloud TPU to fine-tune sentence and sentence-pair classification tasks built on top of pretrained BERT models and 
run predictions on tuned model. The colab demonsrates loading pretrained BERT models from both [TF Hub](https://www.tensorflow.org/hub) and checkpoints.

**Note:**  You will need a GCP (Google Compute Engine) account and a GCS (Google Cloud 
Storage) bucket for this Colab to run.

Please follow the [Google Cloud TPU quickstart](https://cloud.google.com/tpu/docs/quickstart) for how to create GCP account and GCS bucket. You have [$300 free credit](https://cloud.google.com/free/) to get started with any GCP product. You can learn more about Cloud TPU at https://cloud.google.com/tpu/docs.

This notebook is hosted on GitHub. To view it in its original repository, after opening the notebook, select **File > View on GitHub**.

## Learning objectives

In this notebook, you will learn how to train and evaluate a BERT model using TPU.

## Instructions

<h3><a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>  &nbsp;&nbsp;Train on TPU</h3>

   1. Create a Cloud Storage bucket for your TensorBoard logs at http://console.cloud.google.com/storage and fill in the BUCKET parameter in the "Parameters" section below.
 
   1. On the main menu, click Runtime and select **Change runtime type**. Set "TPU" as the hardware accelerator.
   1. Click Runtime again and select **Runtime > Run All** (Watch out: the "Colab-only auth for this notebook and the TPU" cell requires user input). You can also run the cells manually with Shift-ENTER.

### Set up your TPU environment

In this section, you perform the following tasks:

*   Set up a Colab TPU running environment
*   Verify that you are connected to a TPU device
*   Upload your credentials to TPU to access your GCS bucket.

In [3]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.21.128.210:8470


W0813 21:58:41.268386 140613293332352 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 18158176857304413001),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5161139114032175048),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 10048316422614533797),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 10597434460892239874),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 8333610763815774113),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 15818777960531408360),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 10470061386165594979),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 8198994015876653602),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 3547267250771737584),
 _DeviceAttributes(/job:tpu_w

### Prepare and import BERT modules
​
With your environment configured, you can now prepare and import the BERT modules. The following step clones the source code from GitHub and import the modules from the source. Alternatively, you can install BERT using pip (!pip install bert-tensorflow).

In [4]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub 
import tensorflow_hub as hub

Cloning into 'bert_repo'...
remote: Enumerating objects: 333, done.
remote: Total 333 (delta 0), reused 0 (delta 0), pack-reused 333
Receiving objects: 100% (333/333), 279.30 KiB | 3.88 MiB/s, done.
Resolving deltas: 100% (183/183), done.


W0813 21:58:45.353305 140613293332352 deprecation_wrapper.py:119] From bert_repo/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



### Prepare for training

This next section of code performs the following tasks:

*  Specify task and download training data.
*  Specify BERT pretrained model
*  Specify GS bucket, create output directory for model checkpoints and eval results.




In [5]:
TASK = 'MRPC' #@param {type:"string"}
assert TASK in ('MRPC', 'CoLA'), 'Only (MRPC, CoLA) are demonstrated here.'

# Download glue data.
! test -d download_glue_repo || git clone https://gist.github.com/60c2bdb54d156a41194446737ce03e2e.git download_glue_repo
!python download_glue_repo/download_glue_data.py --data_dir='glue_data' --tasks=$TASK

TASK_DATA_DIR = 'glue_data/' + TASK
print('***** Task data directory: {} *****'.format(TASK_DATA_DIR))
!ls $TASK_DATA_DIR

BUCKET = 'datacup-leaders-prize-bucket-1' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/bert-tfhub/models/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)
#print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

Cloning into 'download_glue_repo'...
remote: Enumerating objects: 21, done.
remote: Total 21 (delta 0), reused 0 (delta 0), pack-reused 21
Unpacking objects: 100% (21/21), done.
Processing MRPC...
Local MRPC data not specified, downloading data from https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt
	Completed!
***** Task data directory: glue_data/MRPC *****
dev_ids.tsv  msr_paraphrase_test.txt   test.tsv
dev.tsv      msr_paraphrase_train.txt  train.tsv


Now let's load tokenizer module from TF Hub and play with it.

In [6]:
tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

W0813 21:59:07.642463 140613293332352 deprecation_wrapper.py:119] From bert_repo/run_classifier_with_tfhub.py:151: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0813 21:59:08.057532 140613293332352 deprecation_wrapper.py:119] From bert_repo/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

Also we initilize our hyperprams, prepare the training data and initialize TPU config.

In [7]:
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
MAX_SEQ_LENGTH = 128
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

processors = {
  "cola": run_classifier.ColaProcessor,
  "mnli": run_classifier.MnliProcessor,
  "mrpc": run_classifier.MrpcProcessor,
}
processor = processors[TASK.lower()]()
label_list = processor.get_labels()

# Compute number of train and warmup steps from batch size
train_examples = processor.get_train_examples(TASK_DATA_DIR)
num_train_steps = int(len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Setup TPU related config
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
ITERATIONS_PER_LOOP = 1000

def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))


W0813 21:59:08.222756 140613293332352 deprecation_wrapper.py:119] From bert_repo/run_classifier.py:199: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.



# Fine-tune and Run Predictions on a pretrained BERT Model from TF Hub

This section demonstrates fine-tuning from a pre-trained BERT TF Hub module and running predictions.


In [8]:
# Force TF Hub writes to the GS bucket we provide.
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR

model_fn = run_classifier_with_tfhub.model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  bert_hub_module_handle=BERT_MODEL_HUB
)

estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)


W0813 21:59:09.300712 140613293332352 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7fe2c66babf8>) includes params argument, but params are not passed to Estimator.


At this point, you can now fine-tune the model, evaluate it, and run predictions on it.

In [0]:
# Train the model
def model_train(estimator):
  print('MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...')
  # We'll set sequences to be at most 128 tokens long.
  train_features = run_classifier.convert_examples_to_features(
      train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_examples)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))



In [10]:
model_train(estimator_from_tfhub)

W0813 21:59:09.344632 140613293332352 deprecation_wrapper.py:119] From bert_repo/run_classifier.py:774: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.



MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...
***** Started training at 2019-08-13 21:59:13.175785 *****
  Num examples = 3668
  Batch size = 32
***** Finished training at 2019-08-13 21:59:15.590549 *****


In [0]:
def model_eval(estimator):
  # Eval the model.
  eval_examples = processor.get_dev_examples(TASK_DATA_DIR)
  eval_features = run_classifier.convert_examples_to_features(
      eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(eval_examples)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
  output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  with tf.gfile.GFile(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
      print('  {} = {}'.format(key, str(result[key])))
      writer.write("%s = %s\n" % (key, str(result[key])))


In [12]:
model_eval(estimator_from_tfhub)

***** Started evaluation at 2019-08-13 21:59:16.098329 *****
  Num examples = 408
  Batch size = 8


E0813 21:59:24.212589 140613293332352 tpu.py:376] Operation of type Placeholder (module_apply_tokens/input_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0813 21:59:24.214898 140613293332352 tpu.py:376] Operation of type Placeholder (module_apply_tokens/input_mask) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0813 21:59:24.216434 140613293332352 tpu.py:376] Operation of type Placeholder (module_apply_tokens/segment_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0813 21:59:24.217603 140613293332352 tpu.py:376] Operation of type Placeholder (module_apply_tokens/mlm_positions) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0813 21:59:24.223889 140613293332352 tpu.py:376] Operation of type Placeholder (module_apply_tokens/bert/embeddings/word_embeddings) is not supported on the TPU. Execution will fail if this op is used in the grap

***** Finished evaluation at 2019-08-13 22:00:19.079428 *****
***** Eval results *****
  eval_accuracy = 0.85784316
  eval_loss = 0.6735221
  global_step = 343
  loss = 0.542314


In [0]:
def model_predict(estimator):
  # Make predictions on a subset of eval examples
  prediction_examples = processor.get_dev_examples(TASK_DATA_DIR)[:PREDICT_BATCH_SIZE]
  input_features = run_classifier.convert_examples_to_features(prediction_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
  predictions = estimator.predict(predict_input_fn)

  for example, prediction in zip(prediction_examples, predictions):
    print('text_a: %s\ntext_b: %s\nlabel:%s\nprediction:%s\n' % (example.text_a, example.text_b, str(example.label), prediction['probabilities']))


In [14]:
model_predict(estimator_from_tfhub) 

E0813 22:00:27.133030 140613293332352 tpu.py:376] Operation of type Placeholder (module_apply_tokens/input_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0813 22:00:27.135267 140613293332352 tpu.py:376] Operation of type Placeholder (module_apply_tokens/input_mask) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0813 22:00:27.136950 140613293332352 tpu.py:376] Operation of type Placeholder (module_apply_tokens/segment_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0813 22:00:27.138523 140613293332352 tpu.py:376] Operation of type Placeholder (module_apply_tokens/mlm_positions) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0813 22:00:27.145582 140613293332352 tpu.py:376] Operation of type Placeholder (module_apply_tokens/bert/embeddings/word_embeddings) is not supported on the TPU. Execution will fail if this op is used in the grap

text_a: He said the foodservice pie business doesn 't fit the company 's long-term growth strategy .
text_b: " The foodservice pie business does not fit our long-term growth strategy .
label:1
prediction:[0.00158932 0.99841076]

text_a: Magnarelli said Racicot hated the Iraqi regime and looked forward to using his long years of training in the war .
text_b: His wife said he was " 100 percent behind George Bush " and looked forward to using his years of training in the war .
label:0
prediction:[0.9970925  0.00290753]

text_a: The dollar was at 116.92 yen against the yen , flat on the session , and at 1.2891 against the Swiss franc , also flat .
text_b: The dollar was at 116.78 yen JPY = , virtually flat on the session , and at 1.2871 against the Swiss franc CHF = , down 0.1 percent .
label:0
prediction:[0.10815371 0.8918463 ]

text_a: The AFL-CIO is waiting until October to decide if it will endorse a candidate .
text_b: The AFL-CIO announced Wednesday that it will decide in October whe

# Fine-tune and run predictions on a pre-trained BERT model from checkpoints

Alternatively, you can also load pre-trained BERT models from saved checkpoints.

In [15]:
# Setup task specific model and TPU running config.
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL 
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
!gsutil ls $BERT_PRETRAINED_DIR

CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')

model_fn = run_classifier.model_fn_builder(
  bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
  num_labels=len(label_list),
  init_checkpoint=INIT_CHECKPOINT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  use_one_hot_embeddings=True
)

OUTPUT_DIR = OUTPUT_DIR.replace('bert-tfhub', 'bert-checkpoints')
tf.gfile.MakeDirs(OUTPUT_DIR)

estimator_from_checkpoints = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)

***** BERT pretrained directory: gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12 *****
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_config.json
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.index
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.meta
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/checkpoint
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/vocab.txt


W0813 22:00:49.861065 140613293332352 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7fe2c66baae8>) includes params argument, but params are not passed to Estimator.


Now, you can repeat the training, evaluation, and prediction steps.

In [16]:
model_train(estimator_from_checkpoints)

MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...
***** Started training at 2019-08-13 22:00:53.771423 *****
  Num examples = 3668
  Batch size = 32
***** Finished training at 2019-08-13 22:00:55.229583 *****


In [17]:
model_eval(estimator_from_checkpoints)

***** Started evaluation at 2019-08-13 22:00:55.711062 *****
  Num examples = 408
  Batch size = 8


W0813 22:00:56.232219 140613293332352 deprecation_wrapper.py:119] From bert_repo/modeling.py:490: The name tf.assert_less_equal is deprecated. Please use tf.compat.v1.assert_less_equal instead.

W0813 22:00:56.303525 140613293332352 deprecation.py:323] From bert_repo/modeling.py:671: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0813 22:01:00.135509 140613293332352 deprecation_wrapper.py:119] From bert_repo/run_classifier.py:647: The name tf.trainable_variables is deprecated. Please use tf.compat.v1.trainable_variables instead.

W0813 22:01:00.445033 140613293332352 deprecation_wrapper.py:119] From bert_repo/run_classifier.py:656: The name tf.train.init_from_checkpoint is deprecated. Please use tf.compat.v1.train.init_from_checkpoint instead.

W0813 22:01:01.646464 140613293332352 deprecation_wrapper.py:119] From bert_repo/run_classifier.py:657: The name tf.train.Scaffold is 

***** Finished evaluation at 2019-08-13 22:01:45.105814 *****
***** Eval results *****
  eval_accuracy = 0.8382353
  eval_loss = 0.66811115
  global_step = 343
  loss = 0.7876671


In [88]:
model_predict(estimator_from_checkpoints)

text_a: He said the foodservice pie business doesn 't fit the company 's long-term growth strategy .
text_b: " The foodservice pie business does not fit our long-term growth strategy .
label:1
prediction:[0.0035472 0.9964528]

text_a: Magnarelli said Racicot hated the Iraqi regime and looked forward to using his long years of training in the war .
text_b: His wife said he was " 100 percent behind George Bush " and looked forward to using his years of training in the war .
label:0
prediction:[0.9891451  0.01085493]

text_a: The dollar was at 116.92 yen against the yen , flat on the session , and at 1.2891 against the Swiss franc , also flat .
text_b: The dollar was at 116.78 yen JPY = , virtually flat on the session , and at 1.2871 against the Swiss franc CHF = , down 0.1 percent .
label:0
prediction:[0.97843504 0.02156499]

text_a: The AFL-CIO is waiting until October to decide if it will endorse a candidate .
text_b: The AFL-CIO announced Wednesday that it will decide in October wheth

## What's next

* Learn about [Cloud TPUs](https://cloud.google.com/tpu/docs) that Google designed and optimized specifically to speed up and scale up ML workloads for training and inference and to enable ML engineers and researchers to iterate more quickly.
* Explore the range of [Cloud TPU tutorials and Colabs](https://cloud.google.com/tpu/docs/tutorials) to find other examples that can be used when implementing your ML project.

On Google Cloud Platform, in addition to GPUs and TPUs available on pre-configured [deep learning VMs](https://cloud.google.com/deep-learning-vm/),  you will find [AutoML](https://cloud.google.com/automl/)*(beta)* for training custom models without writing code and [Cloud ML Engine](https://cloud.google.com/ml-engine/docs/) which will allows you to run parallel trainings and hyperparameter tuning of your custom models on powerful distributed hardware.


## Tyler Additions

* Get Leaders Prize data and test model on this data


In [20]:
# Mounts your Google Drive account
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import matplotlib.pyplot as plt

def plot_history(training_history, with_validation):
    # Accuracy Plot
    plt.plot(training_history.history['acc'])
    if with_validation:
        plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    # Loss Plot
    plt.plot(training_history.history['loss'])
    if with_validation:
        plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()
    
    
    
# Normalize all the counts
def normalize_claim_counts(df):
    """
    Strips out extra claims so we have a balanced dataset - i.e. # of labels for 0,1,2 are the same
    """
    true_claims = df[df['label'] == 2]
    neutral_claims = df[df['label'] == 1]
    false_claims = df[df['label'] == 0]
    max_index = min([
        len(true_claims.index),
        len(neutral_claims.index),
        len(false_claims.index),
    ])
    return pd.concat([
        true_claims[0: max_index],
        neutral_claims[0: max_index],
        false_claims[0: max_index]
    ]).sample(frac=1)  # This shuffles

In [0]:
import pandas as pd

# Define variables for filepaths - this separates data source from data processing & later steps
JSON_DATA_PATH = '/content/drive/My Drive/LeadersPrize/Data/json_data.pkl'
ARTICLES_DATA_PATH = '/content/drive/My Drive/LeadersPrize/Data/articles_data.pkl'

json_data = pd.read_pickle(JSON_DATA_PATH)
articles_data = pd.read_pickle(ARTICLES_DATA_PATH)

### Extract a dict of true claims and its related articles

In [0]:
true_claims_to_related_articles = {}
NUM_EXAMPLES = 1000

# Get valid indices, maybe randomize order to get new ones
indices = json_data['related_articles'].keys()

for idx in indices:
  # Only add true statements
  if json_data['label'][idx] == 2:
    true_claims_to_related_articles[json_data['claim'][idx]] = json_data['related_articles'][idx]
    
  # Get NUM_EXAMPLES
  if len(true_claims_to_related_articles.keys()) == NUM_EXAMPLES:
    break

In [248]:
# Print result
for claim, related_articles in true_claims_to_related_articles.items():
  print("Claim: {} \nRelated Articles: {}\n".format(claim, related_articles))

Claim: Maine legislature candidate Leslie Gibson insulted Parkland shooting survivor and activist Emma Gonzalez via Twitter. 
Related Articles: [106868, 127320, 128060]

Claim: In 1988 author Roald Dahl penned an open letter urging parents to have their children vaccinated against measles. 
Related Articles: [123254, 123418, 127464]

Claim: When it comes to fighting terrorism, "Another thing we know that does not work, based on lots of empirical evidence, is torture." 
Related Articles: [41099, 89899, 72543, 82644, 95344, 88361]

Claim: Rhode Island is "almost dead last" among Northeastern states in the length of time first-degree murderers must spend in prison before they’re eligible for parole. 
Related Articles: [8284, 3768, 20091, 82368, 73148, 4493]

Claim: "We are always going to need architects, doctors, we’re going to need professionals with four-year degrees. But 65 to 70 percent of the people in Missouri don’t have degrees." 
Related Articles: [42685, 32007, 33562]

Claim: Th

### Extract a dict of false claims and its related articles

In [0]:
false_claims_to_related_articles = {}
NUM_EXAMPLES = 1000

# Get valid indices, maybe randomize order to get new ones
indices = json_data['related_articles'].keys()

for idx in indices:
  # Only add false statements
  if json_data['label'][idx] == 0:
    false_claims_to_related_articles[json_data['claim'][idx]] = json_data['related_articles'][idx]
    
  # Get NUM_EXAMPLES
  if len(false_claims_to_related_articles.keys()) == NUM_EXAMPLES:
    break

In [250]:
# Print result
for claim, related_articles in false_claims_to_related_articles.items():
  print("Claim: {} \nRelated Articles: {}\n".format(claim, related_articles))

Claim: A line from George Orwell's novel 1984 predicts the power of smartphones. 
Related Articles: [122094, 122580, 130685, 134765]

Claim: Koch Industries paid the legal fees of George Zimmerman. 
Related Articles: [120591, 120592, 127866, 129483]

Claim:  "Pelosi Sinks to New Low, Tells Dems: If You Have to Lie to Voters to Win, Do It"  
Related Articles: [27062, 27061, 20679, 61872, 20677]

Claim: NASA Has Just Confirmed Earth Has A New Moon 
Related Articles: [91455, 72179, 18903, 42080]

Claim: "Justin Amash is rated Michigan’s No. 1 conservative" by National Right to Life. 
Related Articles: [22383, 72467, 72466, 86512, 73422, 83732, 83730]

Claim:  BREAKING: NFL Owner Listens to Trump, Fires Player For Disgracing National Anthem  
Related Articles: [20907, 73380, 22540, 2010]

Claim: Says one year ago, "no cities in the South had guaranteed paid sick days." 
Related Articles: [87410, 18608, 57313, 35767, 85310, 43631]

Claim: Says "the mandate is 71 times that a child’s body wi

### Turn list of related articles to long list of sentences

In [0]:
def get_list_of_related_article_sentences(related_article_nums):
  related_article_sentences = []
  for related_article_num in related_article_nums:
    sentences = []
    article = articles_data['text']['{}'.format(related_article_num)]
    return article_to_sentences(article)

In [0]:
def article_to_sentences(article):
    import re
    # Remove punctuation and line breaks
    sentences = re.split(r"[.!?]", article)
    sentences = [sent.strip(" ") for sent in sentences]
    sentences = [sent.split("\n") for sent in sentences]
    retVal = []
    for sent in sentences:
      for s in sent:
        if len(s) > 0:
          retVal.append(s)
    return retVal

In [0]:
# Break list of article nums to list of sentences
true_claims_to_related_sentences = {}
false_claims_to_related_sentences = {}

for claim, related_article_nums in true_claims_to_related_articles.items():
  true_claims_to_related_sentences[claim] = get_list_of_related_article_sentences(related_article_nums)
for claim, related_article_nums in false_claims_to_related_articles.items():
  false_claims_to_related_sentences[claim] = get_list_of_related_article_sentences(related_article_nums)

In [257]:
# Print result
num_show = 100
i = 0
for claim, related_sentences in true_claims_to_related_sentences.items():
  print("Claim: {} \nRelated Sentences: {}\n".format(claim, related_sentences))
  i += 1
  if i > num_show:
    break
    
# Print result
i = 0
for claim, related_sentences in false_claims_to_related_sentences.items():
  print("Claim: {} \nRelated Sentences: {}\n".format(claim, related_sentences))
  i += 1
  if i > num_show:
    break

Claim: Maine legislature candidate Leslie Gibson insulted Parkland shooting survivor and activist Emma Gonzalez via Twitter. 
Related Sentences: ['Republican who criticized Parkland students drops out of Maine House race', 'Michael Laughlin | AP Michael Laughlin | AP', 'By Caitlin Troutman, Maine Public • March 16, 2018 6:53 pm', 'Updated: March 21, 2018 2:06 pm', 'Republican Leslie Gibson has announced that he is dropping out of the race for Maine House District 57, representing Greene and Sabattus', 'Gibson came under fire for his Twitter remarks about two students who survived the Feb', '14 shooting at Marjory Stoneman Douglas High School in Parkland, Florida', 'He wrote that one of the Parkland students was a “skinhead lesbian,” the other a “bald-faced liar', '” Gibson later apologized to one of the students', '[Maine House candidate calls Parkland student activist ‘skinhead lesbian’]', 'Gibson said he made the decision after meeting with friends, family and colleagues, saying that

### Convert dict to list of prediction_examples

In [0]:
# Conver to InputExamples format
import run_classifier

prediction_examples = []
i = 0
for claim, related_sentences in true_claims_to_related_sentences.items():
  for sentence in related_sentences:
    prediction_examples.append(run_classifier.InputExample(guid=i, text_a=claim, text_b=sentence, label='1'))
    i += 1

for claim, related_sentences in false_claims_to_related_sentences.items():
  for sentence in related_sentences:
    prediction_examples.append(run_classifier.InputExample(guid=i, text_a=claim, text_b=sentence, label='0'))
    i += 1

### Run predictions and see results

In [0]:
def tyler_model_predict(estimator, my_examples):
  # Make predictions on a subset of eval examples
  #prediction_examples = processor.get_dev_examples(TASK_DATA_DIR)[:PREDICT_BATCH_SIZE]
  prediction_examples = my_examples
  input_features = run_classifier.convert_examples_to_features(prediction_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
  predictions = estimator.predict(predict_input_fn)

  return prediction_examples, predictions

In [0]:
prediction_examples, predictions = tyler_model_predict(estimator_from_checkpoints, prediction_examples)

In [0]:
# Convert generator to list for convenience
predictions = list(predictions)

In [184]:
i = 0
for example, prediction in zip(prediction_examples, predictions):
  i += 1
  if i > 100:
    break
  print('text_a: %s\ntext_b: %s\nlabel:%s\nprediction:%s\n' % (example.text_a, example.text_b, str(example.label), prediction['probabilities']))

text_a: Maine legislature candidate Leslie Gibson insulted Parkland shooting survivor and activist Emma Gonzalez via Twitter.
text_b: Republican who criticized Parkland students drops out of Maine House race
label:1
prediction:[0.9879019  0.01209808]

text_a: Maine legislature candidate Leslie Gibson insulted Parkland shooting survivor and activist Emma Gonzalez via Twitter.
text_b: Michael Laughlin | AP Michael Laughlin | AP
label:1
prediction:[0.9857789  0.01422106]

text_a: Maine legislature candidate Leslie Gibson insulted Parkland shooting survivor and activist Emma Gonzalez via Twitter.
text_b: By Caitlin Troutman, Maine Public • March 16, 2018 6:53 pm
label:1
prediction:[0.98781043 0.01218956]

text_a: Maine legislature candidate Leslie Gibson insulted Parkland shooting survivor and activist Emma Gonzalez via Twitter.
text_b: Updated: March 21, 2018 2:06 pm
label:1
prediction:[0.98445565 0.01554433]

text_a: Maine legislature candidate Leslie Gibson insulted Parkland shooting su

In [197]:
true_or_false = {}
for example, prediction in zip(prediction_examples, predictions):
  if not '{}'.format(example.label) + example.text_a in true_or_false:
    print('New: {}'.format(example.label) + example.text_a)
    true_or_false['{}'.format(example.label) + example.text_a] = prediction['probabilities'][1]
  elif true_or_false['{}'.format(example.label) + example.text_a] < prediction['probabilities'][1]:
    print("Update from {} to {}".format(true_or_false['{}'.format(example.label) + example.text_a], prediction['probabilities'][1]))
    true_or_false['{}'.format(example.label) + example.text_a] = prediction['probabilities'][1]
    


New: 1Maine legislature candidate Leslie Gibson insulted Parkland shooting survivor and activist Emma Gonzalez via Twitter.
Update from 0.01209807675331831 to 0.014221064746379852
Update from 0.014221064746379852 to 0.015544326044619083
Update from 0.015544326044619083 to 0.01965240389108658
New: 1In 1988 author Roald Dahl penned an open letter urging parents to have their children vaccinated against measles.
Update from 0.012250957079231739 to 0.01538274995982647
Update from 0.01538274995982647 to 0.01959623210132122
Update from 0.01959623210132122 to 0.023000864312052727
Update from 0.023000864312052727 to 0.0257598627358675
Update from 0.0257598627358675 to 0.9814506769180298
New: 1When it comes to fighting terrorism, "Another thing we know that does not work, based on lots of empirical evidence, is torture."
Update from 0.012226510792970657 to 0.9901060461997986
New: 1Rhode Island is "almost dead last" among Northeastern states in the length of time first-degree murderers must spen

In [195]:
print(len(prediction_examples))
print(len(predictions))


15573
15573


AttributeError: ignored

In [200]:
for example, prediction in zip(prediction_examples, predictions):
  if prediction['probabilities'][1] > 0.5:
    print(example.text_a + "||" + example.text_b + "-|-" + example.label + '\n')

In 1988 author Roald Dahl penned an open letter urging parents to have their children vaccinated against measles.||Over twenty years later, Roald Dahl wrote the following passage, aimed at parents who were refusing to give their children the measles vaccine-|-1

When it comes to fighting terrorism, "Another thing we know that does not work, based on lots of empirical evidence, is torture."||Another thing we know that does not work—based on lots of empirical evidence—is torture-|-1

Says "51 percent of federal tax filers paid zero federal income tax."||As a result, 51% of federal tax filers paid zero federal income tax-|-1

"We spend more money on lobbying than we do on campaigns."||That's why we spend more time on lobbying than we do on campaigns-|-1

Obama has "visited more countries and met with more world leaders than any president in his first six months in office."||Visited more countries and met with more world leaders than any president in his first six months in office-|-1

"Ac